In [17]:
#impoering the libraries and moduels
import torch
import torch.nn as nn
from torch.utils.data import Dataset ,DataLoader
from sklearn.model_selection import train_test_split
from torchsummary import summary
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.optim import Adam

device = 'cuda' if torch.cuda.is_available() else 'cpu'


#importing and cleaning the dataset
test_data_df = pd.read_csv(r"D:\Datasets\Kaggle\Driving _behaviour_dataset\test_motion_data.csv")
train_data_df = pd.read_csv(r"D:\Datasets\Kaggle\Driving _behaviour_dataset\train_motion_data.csv")
original_test_data_df = test_data_df.copy()
original_train_data_df = train_data_df.copy()
test_data_df.dropna(inplace=True)
train_data_df.dropna(inplace=True)


#normalizing 
for column in train_data_df:
    if train_data_df[column].dtype in  ['int','float']:
        train_data_df[column] = train_data_df[column]/train_data_df[column].abs().max()

for column in test_data_df:
    if test_data_df[column].dtype in ['int','float']:
        test_data_df[column] = test_data_df[column]/test_data_df[column].abs().max()

#omitting out the timestamp column cuz its irrelevant
test_data_df.drop(['Timestamp'], axis=1, inplace=True)
train_data_df.drop(['Timestamp'], axis=1, inplace=True)
print(train_data_df.head())

#replacing the class column with integeres witht he help of mapping
class_mapping = {
    'SLOW' : 0,
    'NORMAL' : 1,
    'AGGRESSIVE' : 2
}
train_data_df['Class'] = train_data_df['Class'].replace(class_mapping)
test_data_df['Class'] = test_data_df['Class'].replace(class_mapping)
print(train_data_df['Class'])

       AccX      AccY      AccZ     GyroX     GyroY     GyroZ   Class
0  0.000000  0.000000  0.000000  0.069951 -0.104000  0.082443  NORMAL
1 -0.325915 -0.230327 -0.028581 -0.033627  0.030545  0.109615  NORMAL
2 -0.119277 -0.026046  0.030865 -0.022838 -0.017455  0.071080  NORMAL
3  0.148124 -0.048610  0.093468  0.082179 -0.017818  0.044402  NORMAL
4  0.020407  0.165447 -0.009341  0.036145 -0.002182  0.044402  NORMAL
0       1
1       1
2       1
3       1
4       1
       ..
3639    0
3640    0
3641    0
3642    0
3643    0
Name: Class, Length: 3644, dtype: int64


C:\Users\HP\AppData\Local\Temp\ipykernel_39296\3544475429.py:45: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data_df['Class'] = train_data_df['Class'].replace(class_mapping)
C:\Users\HP\AppData\Local\Temp\ipykernel_39296\3544475429.py:46: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data_df['Class'] = test_data_df['Class'].replace(class_mapping)


## Current Workflow ##


1. Data cleaning - done
2. Dataset setup: most conceptual i believe
3. DataLoader magik
4. Design the model, ooh gawd
5. Training loop ooh lawd have mercy
6. Test the model, ** insert confused cat meme here **

In [18]:
###Dataset magik

#converting to numpy arrays
X = np.array(train_data_df.iloc[:,:-1])
Y = np.array(train_data_df.iloc[:,-1])
X_test = np.array(test_data_df.iloc[:,:-1])
Y_test = np.array(test_data_df.iloc[:,-1])

#creating a sweet validation set from train set
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, train_size=0.8)

class dataset(Dataset):
    def __init__(self, X, Y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.Y = torch.tensor(Y, dtype=torch.long)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.Y[index]
    

In [19]:
training_data = dataset(X_train, Y_train)
validation_data = dataset(X_val, Y_val)
test_data = dataset(X_test, Y_test)

#Loading the dataset into dataloader
train_dataloader = DataLoader(training_data, batch_size=8, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=True)

In [20]:
#Designing the model
class DriveModel(nn.Module):
    def __init__(self):
        super(DriveModel, self). __init__()
        self.input_layer = nn.Linear(X.shape[1], 16)
        self.relu = nn.ReLU()
        self.output_layer = nn.Linear(16,3)

    def forward(self, x):
        x = self.input_layer(x)
        x = self.relu(x)
        output = self.output_layer(x)
        return x
    
model = DriveModel().to(device)


In [23]:
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr = 1e-3)

In [ ]:
#Training loop
epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device),labels.to(device)
        optimizer.zero_grad()
        output = model(inputs)
        batch_loss = loss_fn(output, labels)
        batch_loss.backward()
        optimizer.step()

        running_loss += batch_loss.item()
        avg_train_loss = running_loss/len(train_dataloader)


    #Validation loop
    model.eval()
    val_running_loss = 0
    val_running_loss = 0
    with torch.no_grad():
        for val_input, val_labels in validation_dataloader:
            val_input = val_input.to(device)
            val_labels = val_labels.to(device)

            val_output = DriveModel(val_input)
            val_batch_loss = loss_fn(val_output, val_labels)

            val_running_loss += val_batch_loss.item()
    avg_val_loss = val_running_loss/len(validation_dataloader)


    print('**' * 12)

    print(f"Epoch: [{epoch+1}/{epochs}]")
    print(f"Training loss(avg): {avg_train_loss}")
    print(f"Validation loss(avg): {avg_val_loss}")




    
